In [4]:
import cv2
import tensorflow.keras as keras
import numpy as np

model=keras.models.load_model("./model/facial_model_overfit.hdf5")


classes={0: 'surprise',
         1: 'disgust',
         2: 'fear',
         3: 'neutral',
         4: 'happiness',
         5: 'sadness',
         6: 'anger',
         7: 'contempt'}
         

face_classifier=cv2.CascadeClassifier("./CascadeFile/haarcascade_frontalface_alt.xml")
#eye_classifier=cv2.CascadeClassifier("./CascadeFile/haarcascade_eye.xml")

v_offset=0
h_offset=0

def detect_face(gray_img,frame):
    faces=[]
    face_img=gray_img.copy()
    face_rects=face_classifier.detectMultiScale(face_img,scaleFactor=1.3,minNeighbors=6,minSize=(30, 30)) #1.3,4
    
    for (x,y,h,w) in face_rects:
        faces.append(face_img.copy()[y-v_offset:y+h+v_offset,x-h_offset:x+w+h_offset])
        cv2.rectangle(frame,(x-h_offset,y-v_offset),(x+w+h_offset,y+h+v_offset),255,10)
        #roi=face_img[y:y+h,x:x+w]
        #eye_l=eye_classifier.detectMultiScale(roi)#,scaleFactor=1.2,minNeighbors=5)
        #if len(eye_l)>0:
            #faces.append(roi)
            #cv2.rectangle(frame,(x-h_offset,y-v_offset),(x+w+h_offset,y+h+v_offset),255,10)
    return np.array(faces)
    
    
cap=cv2.VideoCapture("Bella Thorne - Burn so Bright from the Midnight Sun Original Motion Picture Soundtrack.mp4")

fontFace=cv2.FONT_HERSHEY_SIMPLEX

#fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#writer = cv2.VideoWriter('output.mp4', fourcc , 30.0 , (960, 540))


while True:
    
    ret,frame=cap.read()
    frame = cv2.resize(frame, (960, 540))
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=detect_face(gray,frame)
    #operation
    if len(faces)>0:
        im=np.array([cv2.resize(face,(128,128)) for face in faces])
        #im=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        im=im.reshape(-1,128,128,1)/255.0
        pred=model.predict_classes(im)
        cv2.putText(frame,text=str(np.array(list(classes.values()))[pred]),org=(0,100),fontFace=fontFace,fontScale=1,color=(0,0,255),thickness=2)

    #writer.write(frame)
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break
        
cap.release()
#writer.release()
cv2.destroyAllWindows()


#'http://192.168.31.58:8080/video'
#'testVideo.mp4'